<a href="https://colab.research.google.com/github/tomiokario/MathPDF2MD/blob/main/MathPDF2MD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 数式を含むPDFをMarkdownに変換
**概要**
- 数式を含むPDFをOCRしてMarkdownに変換することができます．
- Nougat (Neural Optical Understanding for Academic Documents)[1,2]を使用します．
- 無料版Google Colabで実行できます[3]．

**License**
- Nougat codebase is licensed under [MIT](https://opensource.org/license/mit/) [1,2].
- Nougat model weights are licensed under [CC BY-NC](https://creativecommons.org/licenses/by-nc/4.0/deed.ja) [1,2].


**参考**
1. [Nougat](https://facebookresearch.github.io/nougat/)
2. [facebookresearch/nougat: Implementation of Nougat Neural Optical Understanding for Academic Documents](https://github.com/facebookresearch/nougat)
3. [数式を含むPDFをOCRしてマークダウン形式に変換できる、Nougatを試す｜はまち](https://note.com/hamachi_jp/n/n7f5f35b38768)

---

## I. 前準備
1. ランタイムの設定をGPUに変更
2. PDFファイルをアップロード
  - サイドバー(左)のフォルダメニュー(📁)を開いて，PDFをドラッグアンドドロップ


## II. Nougatのインストール

In [ ]:
!pip install nougat-ocr

## II. Nougatの実行
1. PDFファイルパスを取得
  - サイドバー(左)からフォルダメニュー(📁)を開いて，右端のオプション(縦三点)を押して，「パスをコピー」を押す
2. 以下のスクリプト内部の対応する箇所にペースト
  - 具体的な形式はデフォルトで入力されているサンプルを参照

In [ ]:
!nougat /content/2308.00088.pdf --out .

## IV. Markdownファイルのダウンロード
- サイドバーからファイルメニューを開いて，markdownファイルをダウンロードする．

※ファイルメニューのデータはランタイム終了時に削除されることに注意

## (Option)MakrdownからScrapbox記法に変換する
1. 以下のコードブロックの変数(`input_path`)にmarkdownファイルのパスを指定
2. コードを実行してファイルメニューに出力されるtxtファイルをダウンロード
※実行後，txtファイルへの書き込みが終了するまで十数秒ほどかかります

In [ ]:
input_path = '/content/2308.00088.mmd'  # このパスは適切に設定してください
output_path = input_path+'.txt'  # 出力するファイルのパス

In [ ]:
import re

def markdown_to_scrapbox(line):
    # Heading conversion
    line = re.sub(r'^###### (.+)$', r'[* \1]', line)
    line = re.sub(r'^##### (.+)$', r'[* \1]', line)
    line = re.sub(r'^#### (.+)$', r'[* \1]', line)
    line = re.sub(r'^### (.+)$', r'[** \1]', line)
    line = re.sub(r'^## (.+)$', r'[*** \1]', line)
    line = re.sub(r'^# (.+)$', r'[**** \1]', line)

    # Strong emphasis (bold)
    line = re.sub(r'\*\*(.+?)\*\*', r'[* \1]', line)

    # Links
    line = re.sub(r'\[(.+?)\]\((http[s]?://.+?)\)', r'[\1 \2]', line)
    line = re.sub(r'(http[s]?://.+)', r'[\1]', line)

    # 数式 (例: \( ... \) を [$ ... ] に変換)
    line = re.sub(r'\\\((.+?)\\\)', r'[$ \1]', line)

    # Indentation and Lists (very basic)
    line = re.sub(r'^    - (.+)$', r'      \1', line)
    line = re.sub(r'^   - (.+)$', r'     \1', line)
    line = re.sub(r'^  - (.+)$', r'    \1', line)
    line = re.sub(r'^ - (.+)$', r'   \1', line)
    line = re.sub(r'^- (.+)$', r'  \1', line)

    line = re.sub(r'^    \* (.+)$', r'      \1', line)
    line = re.sub(r'^   \* (.+)$', r'     \1', line)
    line = re.sub(r'^  \* (.+)$', r'    \1', line)
    line = re.sub(r'^ \* (.+)$', r'   \1', line)
    line = re.sub(r'^\* (.+)$', r'  \1', line)

    return line

# 入力ファイルを開く
with open(input_path, 'r', encoding='utf-8') as f:
    array = [line.strip() for line in f.readlines()]


output_string = ""
for line in array:
    line = line.replace('[', '［').replace(']', '］')
    # Convert each line and concatenate to output string
    converted_line = markdown_to_scrapbox(line)
    output_string += converted_line + "\n"

# 変換された内容を出力ファイルに書き込む
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(output_string)